# GERIT PRE-Matching

Im folgenden wird ein Notebook entworfen, dass das derzeitige Matching der HEX der Daten mit GERIT-Organisationen erleichtern soll. Im Derzeitigen Stand geht folgendermaßen vor:

Es wird von einem klassischen Cleaning der Organisations-Variable, wie sie aus den Scrapiung kommt, abgesehen, weil es im vergleich zum folgenden Vorgehen (und angesichts des Umstandes, dass wir ja die *wahren* Organisationennamen schon kennen) uneffizient und die Gefahr von Artfacten höher erscheint.

Statt eines klassischen Cleanings mit Hilfe von LLMs wird eine Funktion implementiert,die nach Entsprechungen in der Liste der GERIT-Organisationen. Dafür werden mit OpenAI Embeddings Vektor-Repräsentationen aller GERIT-Namen erstellt. Ein *„Retriever“* durchsucht diese Vektoren, um die besten Übereinstimmungen zu einem gegebenen Namen (also den Organisationsnamen aus dem Scraping) zu finden. Das Modell kann z. B. erkennen, dass „*Institut für Biologie*“ und „*Department Biologie*“ wahrscheinlich dasselbe meinen.

Für jede Organisation wird überprüft, ob es eine exakte Übereinstimmung gibt. Falls ja, wird der offizielle GERIT-Name übernommen. Falls keine exakte Übereinstimmung gefunden wird, kommt ein Fuzzy Matching zum Einsatz, bei dem ein kosinusbasierter Score berechnet wird (0 = völlig unterschiedliche Begriffe, 1 = perfekte Übereinstimmung). Ist der Score hoch genug (aktuell: > 0.5), wird der bestpassende GERIT-Treffer übernommen. Falls keine passende Entsprechung gefunden wird, bleibt der Wert NA.

Vorteile: Die Funktion spart Zeit – 20.000 Zeilen und 250 unique Werte werden in etwa 1,5 Minuten umcodiert. Zudem ist kein manuelles Coding oder eine zusätzliche Aufbereitung der Organisationsvariablen nötig. Mit dem Score-Cutoff kann weiterhin die Konservativität des tools gesteuert werden.

Nachteile: Die Qualität der Zuordnung muss geprüft werden, und es bleibt die Frage, wie sich das neue >>Cleaning>> oder Vor-Matching auf nachfolgende Analysen auswirkt.



In [2]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching'

Lade Daten

In [ ]:
import pandas as pd

db_hhu = pd.read_csv("data\hhu_db_raw.csv")
#db_hhu = db_hhu.sample(n=200, random_state=42)  # random_state für Reproduzierbarkeit
hhu_gerit = pd.read_excel("data/hhu_gerit.xlsx")

unique values erzeugen

In [ ]:
db_hhu.columns
# Typ der Spalte anzeigen
print(db_hhu["organisation_mehrere"].dtype)

# Einzigartige Werte der Spalte anzeigen
print(db_hhu["organisation_mehrere"].unique())



In [ ]:
import numpy as np

# Sicherstellen, dass die Spalte als String behandelt wird
db_hhu["organisation_mehrere"] = db_hhu["organisation_mehrere"].astype(str)

# Alle Werte splitten, Leerzeichen bereinigen, "NA" entfernen und einzigartige Werte extrahieren
unique_values = set()
db_hhu["organisation_mehrere"].dropna().apply(lambda x: unique_values.update(
    [val.strip() for val in x.split(";") if val.strip() and val.strip().upper() != "NA"]
))

# Ergebnis ausgeben
print(f"Anzahl einzigartiger Werte: {len(unique_values)}")
print(unique_values)

Anzahl einzigartiger Werte: 248
{'Molekulare Enzymtechnologie', 'Institut für Cytopathologie', 'Dekanat Medizinische Fakultät', 'Zell- und Interaktionsbiologie CEPLAS', 'Biochemie II: Biotransformation und Biokatalyse', 'Frauenklinik', 'Klinik und Poliklinik für Psychiatrie und Psychotherapie der Heinrich-Heine-Universität Düsseldorf - LVR-Klinikum Düsseldorf', 'Institut für Laser- und Plasmaphysik', 'Klinik für Kardiovaskuläre Chirurgie', 'Arbeitsgruppe Hochauflösende UV-Laserspektroskopie', 'Abteilung I Mittelalterliche Geschichte', 'Politikwissenschaft III', 'European Studies', 'Philosophische Fakultät allgemein', 'Pflanzliche Zellbiologie und Biotechnologie', 'Abteilung für Politikwissenschaft', 'Klinik für Gastroenterologie, Hepatologie und Infektiologie', 'Physikalische Chemie u. Elektrochemie II', 'Institut für Theoretische Chemie und Computerchemie', 'Philosophische Fakultät', 'Lehrstuhl für Volkswirtschaftslehre, insbes. Monetäre Ökonomik', 'Biochemie der Pflanzen', 'Institut 

### LLM laden



In [ ]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
import openai
import pandas as pd
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain.agents.agent_toolkits import create_retriever_tool

# Funktion zur Erstellung von Embeddings mit dem neuen Modell
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Erstelle InMemoryVectorStore für GERIT-Organisationen
vector_store = InMemoryVectorStore(embeddings)

# Alle Organisationen aus GERIT zu den Vektoren hinzufügen
docs = [Document(page_content=einrichtung) for einrichtung in hhu_gerit["Einrichtung"].tolist()]
vector_store.add_documents(docs)

# Definiere den Retriever als Chain mit Scores
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search_with_score(query, k=5)
    docs, scores = zip(*results) if results else ([], [])

    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score  # Füge den Score als Metadaten hinzu

    return list(docs)

# Beschreibung für den Retriever
description = (
    "Sucht nach gültigen Eigennamen basierend auf einer ungefähren Eingabe und gibt die Ähnlichkeitsscores zurück. "
    "Falls keine passende Entsprechung gefunden wird, wird 'NA' zurückgegeben."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns_with_score",
    description=description,
)


In [ ]:
query = "Chemie"
results = retriever.invoke(query)

# Ergebnisse anzeigen
for doc in results:
    print(f"Text: {doc.page_content}, Score: {doc.metadata['score']}")


Text: Fach Chemie, Score: 0.751009693806074
Text: Fach Physik, Score: 0.5849177282011132
Text: Fach Pharmazie, Score: 0.5565538344507959
Text: Institut für Physikalische Chemie, Score: 0.5558111486761833
Text: Lehrstuhl für Organische Chemie, Score: 0.5419921262512815


In [ ]:
import pandas as pd

def match_unique_organisations(org_series, mapping_dict, retriever):
    """
    Funktion, die nur einzigartige Werte der Organisationen verarbeitet und eine detaillierte Matching-Tabelle erstellt.

    :param org_series: Pandas Series mit den ursprünglichen Organisationen
    :param mapping_dict: Dictionary mit bekannten exakten Zuordnungen
    :param retriever: Retriever zur unscharfen Suche
    :return:
        - recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
        - matching_df: DataFrame mit Matching-Details (ursprünglicher Wert, gematchter Wert, Matching-Art, Score)
    """
    print(f"Anzahl einzigartiger Organisationen in den Eingabedaten: {len(org_series)}")

    recode_dict = {}
    matching_data = []  # Liste für die Matching-Ergebnisse

    for org in org_series:
        orgs = [o.strip() for o in org.split(';')]  # Aufteilen bei ";", Leerzeichen entfernen
        matched_orgs = []
        match_details = []  # Speichert Match-Typ und Score für jede Organisation in einer Zeile

        for o in orgs:
            match_type = "Keine Übereinstimmung"
            score = None
            matched_value = None

            # 1. Prüfen, ob eine exakte Übereinstimmung existiert
            if o in mapping_dict:
                matched_value = mapping_dict[o]
                match_type = "Exakt"
                score = 1.0  # Exakte Matches haben Score 1.0
                print(f"Exakte Übereinstimmung gefunden: '{o}' → '{matched_value}'")
            else:
                # 2. Falls keine exakte Übereinstimmung, unscharfe Suche über den Retriever
                results = retriever.invoke(o)  # invoke nutzen, da retriever eine Chain ist

                if results and results[0].metadata.get("score", 0) >= 0.5:
                    matched_value = results[0].page_content
                    score = results[0].metadata["score"]
                    match_type = "Fuzzy"
                    print(f"Fuzzy Match gefunden: '{o}' → '{matched_value}' (Score: {score:.2f})")
                else:
                    print(f"Kein passender Match für '{o}' gefunden.")

            # Speichern des Matches
            matched_orgs.append(matched_value if matched_value else o)  # Falls kein Match, Original behalten
            match_details.append({
                "Ursprünglicher Wert": o,
                "Gematchter GERIT-Wert": matched_value,
                "Matching-Art": match_type,
                "Score": score
            })

        # Rekodierungs-Dictionary speichern
        recode_dict[org] = "; ".join([m for m in matched_orgs if m]) if matched_orgs else None

        # Matching-Details zur Liste hinzufügen
        matching_data.extend(match_details)

    # DataFrame aus den Matching-Daten erstellen
    matching_df = pd.DataFrame(matching_data)

    return recode_dict, matching_df


In [ ]:
recode_dict, matching_df = match_unique_organisations(unique_values, mapping_dict, retriever)


Anzahl einzigartiger Organisationen in den Eingabedaten: 248
Fuzzy Match gefunden: 'Molekulare Enzymtechnologie' → 'Institut für Molekulare Enzymtechnologie (IMET)' (Score: 0.71)
Fuzzy Match gefunden: 'Institut für Cytopathologie' → 'Institut für Pathologie' (Score: 0.82)
Fuzzy Match gefunden: 'Dekanat Medizinische Fakultät' → 'Institute der Medizinischen Fakultät am Universitätsklinikum Düsseldorf' (Score: 0.64)
Fuzzy Match gefunden: 'Zell- und Interaktionsbiologie CEPLAS' → 'Institut für Zell- und Interaktionsbiologie' (Score: 0.73)
Fuzzy Match gefunden: 'Biochemie II: Biotransformation und Biokatalyse' → 'Lehrstuhl für Biochemie II' (Score: 0.65)
Fuzzy Match gefunden: 'Frauenklinik' → 'Klinik für Frauenheilkunde und Geburtshilfe' (Score: 0.81)
Fuzzy Match gefunden: 'Klinik und Poliklinik für Psychiatrie und Psychotherapie der Heinrich-Heine-Universität Düsseldorf - LVR-Klinikum Düsseldorf' → 'Klinik und Poliklinik für Psychiatrie und Psychotherapie - Rheinische Kliniken Düsseldorf' 

In [ ]:
def apply_matched_organisations(org_series, recode_dict):
    """
    Wendet das zuvor berechnete Matching Dictionary auf die originale Spalte an,
    indem alle Organisationen innerhalb eines Eintrags korrekt ersetzt werden.

    :param org_series: Pandas Series mit den ursprünglichen Organisationen
    :param recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
    :return: Pandas Series mit den rekodierten Organisationen
    """
    def map_multiple_orgs(org):
        if pd.isna(org):
            return None  # Fehlende Werte beibehalten

        org_list = [o.strip() for o in org.split(';')]  # Mehrere Organisationen aufteilen
        matched_list = [recode_dict.get(o, o) for o in org_list]  # Falls kein Match gefunden wird, Originalwert behalten
        matched_list = [m for m in matched_list if m]  # Leere Werte entfernen

        return "; ".join(matched_list) if matched_list else None  # Falls keine Matches, None zurückgeben

    return org_series.apply(map_multiple_orgs)


In [ ]:
db_hhu["gerit_match"] = apply_matched_organisations(db_hhu["organisation_mehrere"], recode_dict)


In [ ]:
matching_df.to_excel("matching_results.xlsx", index=False)